In [323]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn.metrics import classification_report

In [324]:
data_species = pd.read_csv('../Data/species.csv', index_col=0)

data = data_species.copy(deep=True)

#### Remove Duplicates

In [326]:
duplicates = ["REGION", "YEAR", "MAX_HARD_RELIEF", "PCT_CORAL"]

data = data.drop(duplicates, axis = 1)

#### Remove Features not related

In [328]:
not_related = ["no.divers", "model", "Random", "site"]

data = data.drop(not_related, axis = 1)

#### Remove Features not relevant for the model

In [330]:
#Most values are the same for Habitat_type_classLV0
print(data.Habitat_type_classLV0.value_counts())

not_for_model = ["Latitude", "Longitude", "Habitat_type_classLV0"]

data = data.drop(not_for_model, axis = 1)

Habitat_type_classLV0
Coral Reef and Hardbottom    3988
Unconsolidated Sediment         7
Seagrass                        4
Name: count, dtype: int64


## EDA

### Zones and Regions analysis

In [333]:
#Keys_divisions, Region

print(data.loc[data.Keys_Divisions.isna(),["Keys_Divisions", "Region"]])
print(data.loc[data.Keys_Divisions.isna(),["Keys_Divisions", "Region"]].Keys_Divisions.unique())
print(data.loc[data.Keys_Divisions.isna(),["Keys_Divisions", "Region"]].Region.unique())


#CONCLUSION: Kyes_Divisons and Region have the same information. The missing values of Keys_Divisions correspond to SEFCRI.

     Keys_Divisions  Region
1350            NaN  SEFCRI
1351            NaN  SEFCRI
1352            NaN  SEFCRI
1353            NaN  SEFCRI
1354            NaN  SEFCRI
...             ...     ...
3992            NaN  SEFCRI
3993            NaN  SEFCRI
3994            NaN  SEFCRI
3997            NaN  SEFCRI
3998            NaN  SEFCRI

[1429 rows x 2 columns]
[nan]
['SEFCRI']


In [334]:
# Fill missing values for categorical columns with a placeholder
data.fillna({'Keys_Divisions': 'SEFCRI'}, inplace=True)

In [335]:
#BNP, FKNMS, CoralECA, DryTortuga contain the same information combined in Keys_Divisions, so they can be dropped.

drop_individual_region = ["BNP", "FKNMS", "CoralECA", "DryTortugas"]

data = data.drop(drop_individual_region, axis = 1)

### Feature Interrelation Analysis

#### impact

In [338]:
print(data.corr(numeric_only=True).loc[:,["impact", "connectivity"]].sort_values(by = "impact", ascending = False, key = abs))

#CONLCUSION: The following features are highly related and maybe explainable only with the "impact" feature

# SST                            0.848783
# Recreational_fishermen_50km    0.832204
# Commercial_pounds_landed      -0.825649
# Marina_slips_25km              0.817805
# Population_50km                0.817650
# Population_20km                0.770920
# Tourist_fishing                0.741886
# Marina_slips_10km              0.690361
# connectivity                   0.670892

# ** Diversity_index **          0.142606


related_to_impact = ["SST", "Marina_slips_10km", "Marina_slips_25km", 
                     "Population_20km", "Population_50km", "Recreational_fishermen_50km",
                    "Tourist_fishing", "connectivity"]

#OBS: connectivity is also considerably related with the anthropogenic stressors' features


data = data.drop(related_to_impact, axis = 1)

                               impact  connectivity
impact                       1.000000      0.670892
SST                          0.848783      0.550070
Recreational_fishermen_50km  0.832204      0.729267
Commercial_pounds_landed    -0.825649     -0.640719
Marina_slips_25km            0.817805      0.681051
Population_50km              0.817650      0.745041
Population_20km              0.770920      0.687211
Tourist_fishing              0.741886      0.697488
Marina_slips_10km            0.690361      0.766698
connectivity                 0.670892      1.000000
NPP                         -0.663904     -0.556075
Pop_per_area_reef_20km       0.619547      0.650263
Deepwater                   -0.564464     -0.342929
SG_permits_50km              0.486797      0.175019
FSA                         -0.462780     -0.223802
Total_gravity_intercept      0.389253      0.409008
Total_gravity                0.389253      0.409008
Coral_cover                 -0.283849     -0.222864
Artificial_r

#### Depth, Depth_Sbrocco, Deepwater

In [340]:
feat = ["Depth", "Depth_Sbrocco", "Deepwater"]
print(data.corr(numeric_only = True).loc[["Diversity_index"], feat])

#Depth and Depth_Sbrocco have a non-negligible correlation with Diversity_index.

data = data.drop("Deepwater", axis = 1)

                    Depth  Depth_Sbrocco  Deepwater
Diversity_index -0.207735       0.121668   -0.03791


#### Comm_reliance, Comm_engagement, Rec_reliance and Rec_engagement

In [405]:
comm_rec = ["Comm_reliance", "Comm_engagement", "Rec_reliance", "Rec_engagement"]

data.corr(numeric_only = True).loc[:,comm_rec]

print(data["Comm_reliance"].value_counts().head(3))
print(data["Comm_engagement"].value_counts().head(3))
print(data["Rec_reliance"].value_counts().head(3))
print(data["Rec_engagement"].value_counts().head(3))


#CONCLUSION: Most of the values are the same. So they are a good candidate to not be part of the final model. (?)

data = data.drop(comm_rec, axis = 1)

Comm_reliance
-0.129    2433
-0.128     263
 0.347     201
Name: count, dtype: int64
Comm_engagement
-0.158    2433
-0.105     227
 2.960     201
Name: count, dtype: int64
Rec_reliance
-0.185    2736
 0.044     277
 0.995     201
Name: count, dtype: int64
Rec_engagement
-0.260    2736
-0.176     277
 3.101     201
Name: count, dtype: int64


##### Features unsure

In [344]:
"SG_charter_permits_25km", 
"SG_permits_50km",
"Coral_area_UFRTM_20km", 
"FSA", 
"HABITAT_CD",
"NPP",
"Wave_exposure", 
"Artificial_reefs_1km", 
"Year",
"Month"

'Month'

#### EDA above not yet finished (!)

#### Fill Missing Values

In [347]:
#(This will probably give bad results) - For the Coral_cover maybe will be better 
#to remove the rows all together, since we cannot infer by the location



# Handle missing values: fill missing numeric values with the median
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    data[col] = data[col].fillna(data[col].median())

#### Convert Categorical Data into Numeric

In [349]:
# Convert categorical columns to numeric using label encoding
non_numeric_cols = data.select_dtypes(include=['object']).columns
label_encoders = {}
for col in non_numeric_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

In [424]:
data.columns

Index(['Year', 'Month', 'Depth', 'Region', 'Coral_cover', 'Reef_complexity',
       'NPP', 'Wave_exposure', 'Habitat_type_classLV2',
       'Coral_area_UFRTM_20km', 'Coral_area_UFRTM_200km', 'Depth_Sbrocco',
       'FSA', 'Marine_reserve', 'Artificial_reefs_1km', 'SG_permits_50km',
       'SG_charter_permits_25km', 'Total_gravity_intercept', 'Total_gravity',
       'Keys_Divisions', 'Nursery_seagrass', 'Nursery_mangroves',
       'Commercial_pounds_landed', 'Pop_per_area_reef_20km', 'impact',
       'HABITAT_CD', 'Diversity_index'],
      dtype='object')

In [407]:
data.head(10)

,Year,Month,Depth,Region,Coral_cover,Reef_complexity,NPP,Wave_exposure,Habitat_type_classLV2,Coral_area_UFRTM_20km,...,Total_gravity_intercept,Total_gravity,Keys_Divisions,Nursery_seagrass,Nursery_mangroves,Commercial_pounds_landed,Pop_per_area_reef_20km,impact,HABITAT_CD,Diversity_index
0,2005,0,9.800000,1,2.000000,0.550322,550.865051,6.992729,0,88333,...,69.0,69.0,2,6483631.0,199409.31250,1511066.2,0.047128,0.205,8,0.877755
1,2005,3,8.821212,1,6.181818,1.886364,577.431946,6.950751,7,91742,...,74.0,74.0,2,11501368.0,21234.26563,855670.2,0.403795,0.382,18,0.912085
2,2005,2,17.399239,1,7.506345,0.749873,595.320252,6.950181,3,94607,...,126.0,126.0,2,12102752.0,13956.82617,855670.2,0.392888,0.382,2,0.882112
3,2005,0,9.200000,1,10.000000,0.400000,590.443543,6.959807,3,96139,...,126.0,126.0,2,11003196.0,14876.60742,855670.2,0.386919,0.382,8,0.806049
4,2005,3,8.558032,1,2.870486,0.289415,579.874023,6.914992,3,98167,...,126.0,126.0,1,9313051.0,14697.00195,855670.2,0.383887,0.382,8,0.829156
5,2005,2,26.504821,1,17.620519,1.200000,548.318603,7.016505,7,95312,...,126.0,126.0,1,6948246.5,0.00000,855670.2,0.408742,0.382,19,0.826781
6,2005,2,8.696966,1,25.018963,1.149431,547.846313,6.674624,3,101151,...,133.0,133.0,1,10456858.0,187408.04690,855670.2,0.389774,0.382,19,0.934887
7,2005,3,20.300000,1,6.000000,1.300000,539.073120,6.896012,3,100667,...,133.0,133.0,1,10480436.0,162343.26560,855670.2,0.391648,0.382,19,0.853695
8,2005,3,20.463554,1,9.817768,1.700000,548.660706,6.492898,0,103773,...,133.0,133.0,1,12416307.0,271718.68750,855670.2,0.380080,0.382,19,0.837965
9,2005,2,9.500000,1,2.493606,0.849361,551.645569,7.095299,3,105049,...,133.0,133.0,1,13021467.0,296726.59380,855670.2,0.375463,0.382,18,0.890591
